In [ ]:
import galois
import numpy as np

from utils import curve_order, multiply, add, G1, G2, neg, pairing

from py_ecc.fields import optimized_bn128_FQ12 as FQ12

In [ ]:
p = curve_order
FP = galois.GF(p)

In [ ]:
def eval_poly(poly: galois.Poly, at: list):
    poly_G1_terms = [multiply(at[i], int(c)) for i, c in enumerate(poly.coefficients(order="asc"))]
    poly_G1 = poly_G1_terms[0]
    for i in range(1, len(poly_G1_terms)):
        poly_G1 = add(poly_G1, poly_G1_terms[i])
    return poly_G1

## Polynomial commitment

### Single polynom

#### Idea

$P(x) = c_0*x^0 + c_1 * x^1 + ... + c_n * x^n$

$P(\tau) = c_0*\tau^0 + c_1 * \tau^1 + ... + c_n * \tau^n$

$P(r) = c_0*r^0 + c_1 * r^1 + ... + c_n * r^n$

Polynomial Remainder Theorem:

$Q(x) = \frac{P(x) - P(r)}{x-r}$

$Q(\tau) = \frac{P(\tau) - P(r)}{\tau-r}$

---

$(\tau - r)*Q(\tau) = P(\tau) - P(r)$

#### Trusted setup

$CRS = \left\langle [\tau^0]_{G_1}, [\tau^1]_{G_1}], ..., [\tau^n]_{G_1} \right\rangle, [\tau]_{G_2}$

In [ ]:
tau = FP(42)
n = 10

tau_G1 = [multiply(G1, int(tau**i)) for i in range(0, n + 1)]
tau_G2 = multiply(G2, int(tau))

#### Proving

##### Step 1

Given:

$P(x) = c_0*x^0 + c_1 * x^1 + ... + c_n * x^n$

The prover calculates the polynomial commitment for $P(x)$ at $\tau$:

$[P(\tau)]_{G_1} = c_0*[\tau^0]_{G_1} + c_1 * [\tau^1] + ... + c_n * [\tau^n] \color{Cyan} = \left[ c_0*\tau^0 + c_1 * \tau^1 + ... + c_n * \tau^n\\ \right] * G_1$

The prover sends $[P(\tau)]_{G_1}$ to the verifier

In [ ]:
c = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

P = galois.Poly(c, field=FP, order="asc")

assert P.degree <= n, "P.degree > n"

P_tau_G1 = eval_poly(P, tau_G1)

print(f"[P(tau)]G1 = {P_tau_G1}")

##### Step 2

The verifier sends to the prover a random $r$

##### Step 3

The prover calculates:

$[P(r)]_{G_1} = P(r) * G_1$

$[r]_{G_1} = r * G_1$

$[Q(\tau)]_{G_1} = \frac{[P(\tau)]_{G_1} - [P(r)]_{G_1}}{[\tau]_{G_1} - [r]_{G_1}}$

The prover sends $[P(r)]_{G_1}$ and $[Q(\tau)]_{G_1}$

In [ ]:
r = 1337

P_r = P(r)

P_r_G1 = multiply(G1, int(P_r))
r_G1 = multiply(G1, int(r))

Q = (P - P_r) // galois.Poly([1, p-r], field=FP)

Q_tau_G1 = eval_poly(Q, tau_G1)

print(f"[P(r)]G1 = {P_r_G1}")
print(f"[Q(tau)]G1 = {Q_tau_G1}")

#### Verification

The verifier calculates:

$[r]_{G_2} = r * G_2$

$e([Q(\tau)]_{G_1}, [\tau]_{G_2} - [r]_{G_2}) = e([P(\tau)]_{G_1} - [P(r)]_{G_1}, G_2)$

$\color{Cyan}[G_T]^{Q(\tau) * (\tau - r)} = [G_T]^{P(\tau) - P(r)}$

$\color{Cyan}Q(\tau) * (\tau - r) = P(\tau) - P(r)$

$\color{Cyan}Q(\tau) = \frac{P(\tau) - P(r)}{\tau - r}$

In [ ]:
r_G2 = multiply(G2, int(r))

e1 = pairing(add(tau_G2, neg(r_G2)), Q_tau_G1)
e2 = pairing(G2, add(P_tau_G1, neg(P_r_G1)))

assert e1 == e2, "wrong pairing"

### Batched

#### Idea

$f(x) = \sum_{i=[t]}^{}f_i(x)$

$\left\{ cm_i \right\}_{i\in[t]} = f_i(\tau)$

$\left\{ s_i \right\}_{i\in[t]} = f_i(z)$

$h(x) = \sum_{i=1}^{t}\gamma^{i-1}\frac{f_i(x) - f_i(z)}{x - z}$

$W = h(\tau)$

$F = \sum_{i\in[t]}^{}\gamma^{i-1}f_i(\tau)$

$v = \sum_{i\in[t]}^{}\gamma^{i-1}f_i(z)$

---

$(F - u) +(-W \cdot (\tau - z)) = 0$

$(F - u) = W \cdot (\tau - z)$

$\color{Cyan}f(x) - f(z) \equiv  H(x)(X-z)$

$\left( \sum_{i\in[t]}^{}\gamma^{i-1}f_i(\tau)) - \sum_{i\in[t]}^{}\gamma^{i-1}f_i(z) \right) = \left( (\tau - z)\cdot\sum_{i=1}^{t}\gamma^{i-1}\frac{f_i(\tau) - f_i(z)}{\tau - z} \right)$

$\left( \sum_{i\in[t]}^{}\gamma^{i-1}f_i(\tau)) - \sum_{i\in[t]}^{}\gamma^{i-1}f_i(z) \right) = \left( \sum_{i=1}^{t}\gamma^{i-1}f_i(\tau) - \sum_{i=1}^{t}\gamma^{i-1}f_i(z) \right)$

In [ ]:
t = 3
n = 10

tau = 42
z = 1337

P = [galois.Poly(FP.Random(10), field=FP) for _ in range(0, t)]

cm = []
s = []
for i in range(0, t):
    cm.append(P[i](tau))
    s.append(P[i](z))

gamma = 71

H = P[0] - P[0](z)
for i in range(1, t):
    H += (gamma**i * (P[i] - P[i](z)))
H //= galois.Poly([1, p-z], field=FP)

W_tau = H(tau)

f = cm[0]
v = s[0]
for i in range(1, t):
    f += (gamma**i * cm[i])
    v += (gamma**i * s[i])

e1 = f - v
e2 = (tau - z) * (-W_tau)

assert e1 + e2 == 0

#### Proving

##### Step 1

Given:

$f(x) = \sum_{i\in[t]}^{}f_i(x)$

The prover calculates the polynomial commitment for $f_i(x)$ at $\tau$:

$\left\{ [cm_i]_{G_1} \right\}_{i\in[t]} = \sum_{j=0}^{d-1} a_{i,j}[\tau^j]_{G_1} \color{Cyan}= f_i(\tau)$

The prover sends $\{[{f_i(\tau)}]_{G_1}\}_{i\in[t]}$ to the verifier

In [ ]:
t = 3
n = 10

P = [galois.Poly(FP.Random(10), field=FP) for _ in range(0, t)]

cm = [eval_poly(P[i], tau_G1) for i in range(0, t)]

##### Step 2

The verifier sends to the prover a random $z$, $\gamma$

##### Step 3

The prover calculates:

$\left\{ [s_i]_{G_1} \right\}_{i\in[t]} = f_i(z) * G_1$

$h(x) = \sum_{i=1}^{t}\gamma^{i-1}\frac{f_i(x) - f_i(z)}{x - z}$

$[W]_{G_1} = \sum_{i=0}^{d-1} h_i[\tau^j]_{G_1} = \color{Cyan}h(\tau) * G_1$

The prover sends $[W]_{G_1}$

In [ ]:
z = 1337
gamma = 71

s = [multiply(G1, int(P[i](z))) for i in range(0, t)]

H = (P[0] - P[0](z)) // galois.Poly([1, p-z], field=FP)
for i in range(1, t):
    H += gamma**i * ((P[i] - P[i](z)) // galois.Poly([1, p-z], field=FP))

W_G1 = eval_poly(H, tau_G1)


#### Verification

$[F]_{G_1} = \sum_{i\in[t]}^{}\gamma^{i-1}[cm_i]_{G_1} \color{Cyan} = \sum_{i\in[t]}^{}\gamma^{i-1}f_i(\tau)$

$[v]_{G_1} = \sum_{i\in[t]}^{}\gamma^{i-1}[s_i]_{G_1} \color{Cyan}= \sum_{i\in[t]}^{}\gamma^{i-1}f_i(z)$

---

$e([F]_{G_2} - [v]_{G_1}, G2)\cdot e(-[W]_{G_1}, [\tau]_{G_2}-[z]_{G_2}) = 1$

$[G_T]^{(F-u)-W(\tau - z)}=[G_T]^0$

$(F-u) - W(\tau - z) = 0$

$(F-u) = W(\tau - z)$

In [ ]:
f_g1 = cm[0]
for i in range(1, t):
    f_g1 = add(f_g1, multiply(cm[i], gamma**i))

v_g1 = s[0]
for i in range(1, t):
    v_g1 = add(v_g1, multiply(s[i], gamma**i))

z_g2 = multiply(G2, int(z))

e1 = pairing(G2, add(f_g1, neg(v_g1)))
e2 = pairing(add(tau_G2, neg(z_g2)), neg(W_G1))

assert e1 * e2 == FQ12.one()

### Multiple evaluation points

$f(x) = \sum_{i=[t]}^{}f_i(x)$

$f'(x) = \sum_{i=[t']}^{}f'_i(x)$

$\left\{ cm_i \right\}_{i\in[t]} = f_i(\tau)$

$\left\{ cm'_i \right\}_{i\in[t']} = f'_i(\tau)$

$\left\{ s_i \right\}_{i\in[t]} = f_i(z)$

$\left\{ s'_i \right\}_{i\in[t']} = f'_i(z')$

Pick random  $\{v\}_{[t]}$, $\{v'\}_{[t']}$

$h(x) = \sum_{i=1}^{t} v^{i-1} f_i(x) - f_i(z)$

$h'(x) = \sum_{i=1}^{t'} v'^{i-1} f'_i(x) - f'_i(z')$

$W(x) = \frac{h(x)}{x - z}$

$W'(x) = \frac{h'(x)}{x - z'}$

---

Pick random $u$

$F = h(\tau) + uh'(\tau)$

$F + zW(\tau) + uz'W'(\tau) = \tau (W(\tau) + u W'(\tau))$

---

$F + zW(\tau) + uz'W'(\tau) - \tau W(\tau) - r'\tau W'(\tau) = 0$

$F - (\tau - z)W(\tau) - (\tau - z')uW'(\tau) = 0$

$h(\tau) + uh'(\tau) - (\tau - z)\frac{h(\tau)}{\tau - z} - (\tau - z')u\frac{h'(\tau)}{\tau - z'} = 0$

$h(\tau) + uh'(\tau) - h(\tau) - uh'(\tau) = 0 \implies 0=0$

In [ ]:
t1 = 3
t2 = 5

n = 10

z1 = 1337
z2 = 2032

P1 = [galois.Poly(FP.Random(10), field=FP) for _ in range(0, t1)]
P2 = [galois.Poly(FP.Random(10), field=FP) for _ in range(0, t2)]

cm1 = [P1[i](tau) for i in range(0, t1)]
s1 = [P1[i](z1) for i in range(0, t1)]

cm2 = [P2[i](tau) for i in range(0, t2)]
s2 = [P2[i](z2) for i in range(0, t2)]

gamma1 = 71
gamma2 = 37

H1 = P1[0] - P1[0](z1)
for i in range(1, t1):
    H1 += (gamma1**i * (P1[i] - P1[i](z1)))
H1 //= galois.Poly([1, p-z1], field=FP)

H2 = P2[0] - P2[0](z2)
for i in range(1, t2):
    H2 += (gamma2**i * (P2[i] - P2[i](z2)))
H2 //= galois.Poly([1, p-z2], field=FP)

W1_tau = H1(tau)
W2_tau = H2(tau)

r = 76

f1 = cm1[0] - s1[0]
for i in range(1, t1):
    f1 += (gamma1**i * cm1[i]) - (gamma1**i * s1[i])

f2 = cm2[0] - s2[0]
for i in range(1, t2):
    f2 += (gamma2**i * cm2[i]) - (gamma2**i * s2[i])

f = f1 + r * f2

e1 = f + z1 * W1_tau + r*z2 * W2_tau
e2 = (-W1_tau - r*W2_tau)*tau

assert e1 + e2 == 0

In [ ]:
n = 10

P1 = [galois.Poly(FP.Random(10), field=FP) for _ in range(0, t1)]
P2 = [galois.Poly(FP.Random(10), field=FP) for _ in range(0, t2)]

cm1 = [eval_poly(P1[i], tau_G1) for i in range(0, t1)]
s1 = [multiply(G1, int(P1[i](z1))) for i in range(0, t1)]

cm2 = [eval_poly(P2[i], tau_G1) for i in range(0, t2)]
s2 = [multiply(G1, int(P2[i](z2))) for i in range(0, t2)]

In [ ]:
H1 = (P1[0] - P1[0](z1))
for i in range(1, t1):
    H1 += gamma1**i * (P1[i] - P1[i](z1))
H1 //= galois.Poly([1, p-z1], field=FP)

H2 = (P2[0] - P2[0](z2))
for i in range(1, t2):
    H2 += gamma2**i * (P2[i] - P2[i](z2))
H2 //= galois.Poly([1, p-z2], field=FP)

W1_G1 = eval_poly(H1, tau_G1)
W2_G1 = eval_poly(H2, tau_G1)

In [ ]:
f1_g1 = add(cm1[0], neg(s1[0]))
for i in range(1, t1):
    f1_g1 = add(f1_g1, multiply(cm1[i], gamma1**i))
    f1_g1 = add(f1_g1, neg(multiply(s1[i], gamma1**i)))

f2_g1 = add(cm2[0], neg(s2[0]))
for i in range(1, t2):
    f2_g1 = add(f2_g1, multiply(cm2[i], gamma2**i))
    f2_g1 = add(f2_g1, neg(multiply(s2[i], gamma2**i)))

f_g1 = add(f1_g1, multiply(f2_g1, r))

e1 = pairing(G2, add(add(f_g1, multiply(W1_G1, z1)), multiply(W2_G1, r*z2)))
e2 = pairing(tau_G2, add(neg(W1_G1), neg(multiply(W2_G1, r))))

assert e1 * e2 == FQ12.one()

## Vanishing polynomial

To encode some state we need a polynomial $P(x)$ of degree $d$ to evaluate at some 1..n x's to 0

$P(x) = c_0x^0 + c_1x^1 + ... + c_dx^d$

$P(1) = 0$
$P(2) = 0$
$...$
$P(n) = 0$

The vanishing polynomial can be applied to check polynomial evaluation:

$Z_H(x) = \prod_{i=1}^{n}(x-i) = (x-1)(x-2)...(x-n)$

If $Z_H(x)$ divides $P(x)$ without remainder it means that $P(x)$ indeed consists of all required roots

Example:

$P(x) = 16x^8 + 69x^7 + 15x^6 + 25x^5 + 21x^4 + 10x^3 + 33x^1 + 24$

$Z_H(x) = (x-1)(x-2)(x-3)(x-4)(x-5)$

$\frac{P(x)}{Z_H(x)} = \frac{16x^8 + 69x^7 + 15x^6 + 25x^5 + 21x^4 + 10x^3 + 33x^1 + 24}{(x-1)(x-2)(x-3)(x-4)(x-5)}$

$\frac{P(x)}{Z_H(x)} = \frac{(x-1)(x-2)(x-3)(x-4)(x-5)(16x^3 + 25x^2 + 24x + 14)}{{(x-1)(x-2)(x-3)(x-4)(x-5)}} = 16x^3 + 25x^2 + 24x + 14$

In [ ]:
p = 71
Fp = galois.GF(p)
n = 5

P = galois.Poly([16, 69, 15, 25, 21, 10,  0, 33, 24], field=Fp)

Z_H = galois.Poly([1, p-1], field=Fp)
for i in range(2, n + 1):
    Z_H *= galois.Poly([1, p-i], field=Fp)

rem = P % Z_H

assert rem == 0, "P(x) is not divisible by Z_H(x)"

for i in range(1, n):
    assert P(i) == 0, f"P({i}) is not zero"

#### Optimization

Within $\mathbb{F}_p$, there must be $\omega$ such that $\omega^n = 1$

Example:

$n = 8$
$p = 73$
$\omega = 10$

$\omega$ is a generator of subgroup $\mathbb{F}_p$ with order = n, or the primitive n-th root of unity in the finite field

$\left\langle \omega^i \right\rangle_{i\in\{0..n-1\}} = \left\langle 1, 10, 27, 51, 72, 63, 46, 22 \right\rangle$ - roots of unity

In [ ]:
p = 73
Fp = galois.GF(p)

n = 8

omega = Fp.primitive_root_of_unity(n)
roots = [omega**i for i in range(n)]

print(roots)

Using the roots of unity the vanishing polynomial $Z_H(x)$ can be optimized:

$\left\langle \omega^i \right\rangle_{i\in\{0..n-1\}}$

$Z_H(x) = \prod_{i=0}^{n-1}(x-w^i) = (x-\omega^0)(x-\omega^1)...(x-\omega^{n-1})=x^n -1$

So encoding a program into P(x) polynomial at x points that are equal to the roots of unity (instead of {1..d}), $Z_H(x)$ vanishing polynomial can be computed very fast

In [ ]:
Z_H_1 = galois.Poly([1, roots[0]], field=Fp)
for i in range(1, len(roots)):
    Z_H_1 *= galois.Poly([1, roots[i]], field=Fp)

Z_H_2 = galois.Poly([1] + [0 for _ in range(n-1)] + [p-1], field=Fp)

assert Z_H_1 == Z_H_2

## Gate constraints

We want to prove that we know such $x, y$ that the equation $out = 2x^2 - x^2y^2 + 3$ holds

$x = 2$
$y = 3$
$out = 2x^2 - x^2y^2 + 3$

The equation should be converted to gates:

<img src="./images/gates.webp" width="50%" />

$g_i = q_{L_i} * a_i + q_{R_i} * b_i + q_{M_i} * (a_ib_i) + q_{C_i} + q_{O_i} * c_i = 0$

$\left\{ 
\begin{array}{l}
g_0 = 0 * a_0 + 0 * b_0 + 1 * (a_0b_0) + 0 + (-1) * c_0 = 0 & \color{Cyan} g_0 = xx\\
g_1 = 0 * a_1 + 0 * b_1 + 1 * (a_1b_1) + 0 + (-1) * c_1 = 0 & \color{Cyan} g_1 = xx\\
g_2 = 0 * a_2 + 0 * b_2 + 1 * (a_2b_2) + 0 + (-1) * c_2 = 0 & \color{Cyan} g_2 = yy\\
g_3 = 2 * a_3 + 0 * b_3 + 0 * (a_3b_3) + 0 + (-1) * c_3= 0 & \color{Cyan} g_3 = 2g_0\\
g_4 = 0 * a_4 + 0 * b_4 + 1 * (a_4b_4) + 0 + (-1) * c_4 = 0 & \color{Cyan} g_4 = g_1g_2\\
g_5 = 1 * a_5 + (-1) * b_5 + 0 * (a_5b_5) + 0 + (-1) * c_5 = 0 & \color{Cyan} g_5 = g_3+g_4\\
g_6 = 1 * a_6 + 0 * b_6 + 0 * (a_6b_6) + 3 + (-1) * c_6 = 0 & \color{Cyan} g_6 = g_5+3\\
\end{array}
\right.$

$\begin{matrix}
 & \color{Cyan} a & \color{Cyan} b & \color{Cyan} c & \color{Cyan} q_{L_i} & \color{Cyan} q_{R_i} & \color{Cyan} q_{M_i} & \color{Cyan} q_{C_i} & \color{Cyan} q_{O_i} \\
\color{Cyan} g_0 & 2 & 2 & 4 & 0 & 0 & 1 & 0 & -1 \\
\color{Cyan} g_1 & 2 & 2 & 4 & 0 & 0 & 1 & 0 & -1 \\
\color{Cyan} g_2 & 3 & 3 & 9 & 0 & 0 & 1 & 0 & -1 \\
\color{Cyan} g_3 & 4 & 0 & 8 & 2 & 0 & 0 & 0 & -1 \\
\color{Cyan} g_4 & 4 & 9 & 36 & 0 & 0 & 1 & 0 & -1 \\
\color{Cyan} g_5 & 8 & 36 & -28 & 1 & -1 & 0 & 0 & -1 \\
\color{Cyan} g_6 & -28 & 3 & -25 & 1 & 0 & 0 & 3 & -1 \\
\color{Cyan} g_7 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
\end{matrix}$

In [ ]:
def pad_array(a, n):
    return a + [0]*(n - len(a))

# required amount of gates (the nearest power of 2 greater than actual amount of gates)
n = 8

# witness vectors
a = [2, 2, 3, 4, 4, 8, -28]
b = [2, 2, 3, 0, 9, 36, 3]
c = [4, 4, 9, 8, 36, -28, -25]

# gate vectors
ql = [0, 0, 0, 2, 0, 1, 1]
qr = [0, 0, 0, 0, 0, -1, 0]
qm = [1, 1, 1, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3]
qo = [-1, -1, -1, -1, -1, -1, -1]

# pad vectors to length n
a = pad_array(a, n)
b = pad_array(b, n)
c = pad_array(c, n)
ql = pad_array(ql, n)
qr = pad_array(qr, n)
qm = pad_array(qm, n)
qc = pad_array(qc, n)
qo = pad_array(qo, n)

for i in range(n):
    assert a[i]*ql[i] + b[i]*qr[i] + a[i]*b[i]*qm[i] + c[i]*qo[i] + qc[i] == 0, f"Gate {i} is not satisfied"

## Permutation check

### Idea

There are two lists $a$ and $b$. We want to check if they consists of the same values

$a = \left\langle 1,2,3,4 \right\rangle$
$b = \left\langle 1,2,4,3 \right\rangle$

The following equation must hold if they do:

$\prod_{i=0}^{n-1}\frac{a_i}{b_i} = 1$

$\frac{1}{1} * \frac{3}{2} * \frac{2}{4} * \frac{4}{3} = 1$

To avoid cases like the following one:

$a = \left\langle 1,3,6 \right\rangle$
$b = \left\langle 1,2,9 \right\rangle$

$\frac{1}{1} * \frac{3}{2} * \frac{6}{9} = 1$

We should add random shift $\lambda$:

$\prod_{i=0}^{n-1}\frac{a_i + \lambda}{b_i + \lambda} = 1$

$\lambda = 4$

$\frac{1 + 4}{1 + 4} * \frac{3 + 4}{2 + 4} * \frac{6 + 4}{9 + 4} = 0.89 \not= 1$

We need to associate values their corresponding wire indexes.

$a = \left\langle 1,3,2,4 \right\rangle$

$b = \left\langle 1,2,4,3 \right\rangle$

$\sigma(i_b) = i_a$

The $\sigma$ function maps indexes from b to a

$\sigma(1) = 1$ - 1th element of $b$ is at 1th position in $a$

$\sigma(2) = 3$ - 2th element of $b$ is at 3th position in $a$

$\sigma(3) = 4$ - 3th element of $b$ is at 4th position in $a$

$\sigma(4) = 2$ - 4th element of $b$ is at 2th position in $a$

We can encode $a$ and $b$ into tuples $(p, q)$:

$a = [(a_0, 1), (a_1, 2), (a_2, 3), (a_3, 4)] = [(1, 1), (3, 2), (2, 3), (4, 4)]$

$b = [(b_0, \sigma(1)), (b_1, \sigma(2)), (b_2, \sigma(3)), (b_3, \sigma(4))] = [(1, 1), (2, 3), (4, 4), (2, 3)]$

And convert it as $p + q\beta$, $\beta$ - random value:

$a = [a_0 + 1\beta, a_1 + 2\beta, a_2 + 3\beta, a_3 + 4\beta] = [1 + 1\beta, 3 + 2\beta, 2 + 3\beta, 4 + 4\beta]$

$b = [b_0 + \sigma(1)\beta, b_1 + \sigma(2)\beta, b_2 + \sigma(3)\beta, b_3 + \sigma(4)\beta] = [1 + 1\beta, 2 + 3\beta, 4 + 4\beta, 2 + 3\beta]$

Applying the equation:

$\frac{1 + 1\beta + \gamma}{1 + 1\beta + \gamma}*\frac{3 + 2\beta + \gamma}{2 + 3\beta + \gamma}*\frac{2 + 3\beta + \gamma}{4 + 4\beta + \gamma}*\frac{4 + 4\beta + \gamma}{3 + 2\beta + \gamma} = 1$

Selector and witness vectors are not enough to prove the statement. We need to prove interconnection between gates using permutation check.

<img src="./images/gates2.png" width="50%" />

Permutations:

$\begin{matrix}
\color{cyan} a & \color{cyan} b & \color{cyan} c \\
a_0 \to a_2 & b_0 \to b_1 & c_0 \to a_1 \\
a_1 \to с_0 & b_1 \to a_0 & c_1 \to b_2 \\
a_2 \to b_0 & b_2 \to c_1 & c_2 \to a_3 \\
a_3 \to c_2 & b_3 \to b_3 & c_3 \to c_3 \\
\end{matrix}$

To build the constraint we need all witnesses to have unique index.

Assume that the permutations indexed as:

$a: i \to i$
$b: i \to i + n$
$c: i \to i + 2n$

$\sigma(i): \begin{matrix}
\color{cyan} a & \color{cyan} b & \color{cyan} c \\
0 \to 2 & 4 \to 5 & 8 \to 1 \\
1 \to 8 & 5 \to 0 & 9 \to 6 \\
2 \to 4 & 6 \to 9 & 10 \to 3 \\
3 \to 10 & 7 \to 7 & 11 \to 11 \\
\end{matrix}$

The permutation check:

$\prod_{i=0}^{n-1}\frac{a_i + i\beta + \gamma}{a_i + \sigma(i) + \gamma}*\frac{b_i + (i+n)\beta+\gamma}{b_i+\sigma(i+n)\beta+\gamma}*\frac{c_i+(i+2n)\beta+\gamma}{c_i+\sigma(i+2n)+\gamma} = 1$

In [ ]:
n = 4

p = 97
Fp = galois.GF(p)

a = Fp([4, 16, 4, 68])
b = Fp([4, 4, 64, 68])
c = Fp([16, 64, 68, 73])

ai = range(0, n)
bi = range(n, 2*n)
ci = range(2*n, 3*n)

sigma = {
    ai[0]: ai[2], ai[1]: ci[0], ai[2]: bi[0], ai[3]: ci[2],
    bi[0]: bi[1], bi[1]: ai[0], bi[2]: ci[1], bi[3]: bi[3],
    ci[0]: ai[1], ci[1]: bi[2], ci[2]: ai[3], ci[3]: ci[3],
}

gamma = Fp(10)
beta = Fp(4)

acc = Fp(1)
for i in range(n):
    acc *= (a[i] + beta * i + gamma) / (a[i] + beta * sigma[i] + gamma)
    acc *= (b[i] + beta * (i + n) + gamma) / (b[i] + beta * sigma[i + n] + gamma)
    acc *= (c[i] + beta * (i + 2 * n) + gamma) / (c[i] + beta * sigma[i + 2 * n] + gamma)

assert acc == 1, "Accumulator is not equal to 1"

Since we need to encode this constraint as a polynomial, it's better to use indexes generated by $\omega$ roots of unity instead of as in the example above.

We need to map all witnesses to unique indexes using $\omega$ generator:

$H = \left\langle \omega^i \right\rangle_{i\in\{0..n-1\}}$

$a: i \to H_i$
$b: i \to k_1H_i$
$c: i\to k_2H_i$

The constant $k_1$ is chosen so that it is not an element of $H$, and $k_2$ is chosen so that it is neither an element of $H$ nor $k_1H$.

$\begin{matrix}
\color{Cyan}\text{el} & \color{Cyan}a_0 & \color{Cyan}a_1 & \color{Cyan}a_2 & \color{Cyan}a_3 & & \color{Cyan}b_0 & \color{Cyan}b_1 & \color{Cyan}b_2 & \color{Cyan}b_3 &  & \color{Cyan}c_0 & \color{Cyan}c_1 & \color{Cyan}c_2 & \color{Cyan}c_3 \\
\color{Cyan}\text{value} & 4 & 16 & 4 & 68 & & 4 & 4 & 64 & 5 & & 16 & 64 & 68 & 73 \\
\color{Cyan}i & 0 & 1 & 2 & 3 & & 4 & 5 & 6 & 7 &  & 8 & 9 & 10 & 11 \\
\color{Cyan}\sigma(i) & 2 & 8 & 4 & 10 &  & 5 & 0 & 9 & 7 &  & 1 & 6 & 3 & 11 \\
\color{Cyan}H & 1 & 22 & 96 & 75 & \color{Cyan}k_1H & 2 & 44 & 95 & 53 & \color{Cyan}k_2H & 3 & 66 & 94 & 31 \\
\color{Cyan}\sigma(H) & 96 & 3 & 2 & 94 & \color{Cyan}\sigma(k_1H) & 44 & 1 & 66 & 53 & \color{Cyan}\sigma(k_2H) & 22 & 95 & 75 & 31 \\
\end{matrix}$

$\prod_{i=0}^{n-1}\frac{a_i + \beta\omega^i + \gamma}{a_i + \beta\sigma(\omega^i) + \gamma}\cdot\frac{b_i + \beta k_1 \omega^i+\gamma}{b_i+\beta\sigma(k_1\omega^i)+\gamma}\cdot\frac{c_i+\beta k_2 \omega^i+\gamma}{c_i+\beta\sigma(k_2\omega^i)+\gamma} = 1$

$\frac{4 + 1}{4+96}\frac{16 + 22}{16+3}\frac{4+96}{4+2}\frac{68+75}{68+94}\frac{4+2}{4+44}\frac{4+44}{4+1}\frac{64+95}{64+66}\frac{5+53}{5+53}\frac{16+3}{16+22}\frac{64+66}{64+95}\frac{68+94}{68+75}\frac{73+31}{73+31} = 1$, where $\gamma = 0, \beta = 1$

To make this constraint provable in the following protocol, we need to extract sigma functions:

$\sigma_1 = \{\sigma(H)\} = \{96, 3, 2, 94\}$

$\sigma_2 = \{\sigma(k_1H)\} = \{44, 1, 66, 53\}$

$\sigma_3 = \{\sigma(k_2H)\} = \{22, 95, 75, 31\}$

$S_{\sigma_1}(x): H \to \sigma_1$

$S_{\sigma_2}(x): H \to \sigma_2$

$S_{\sigma_3}(x): H \to \sigma_3$

$\prod_{i=0}^{n-1}\frac{a_i + \beta\omega^i + \gamma}{a_i + \beta S_{\sigma_a}(\omega^i) + \gamma}\cdot\frac{b_i + \beta k_1 \omega^i+\gamma}{b_i+\beta S_{\sigma_b}(\omega^i)+\gamma}\cdot\frac{c_i+\beta k_2 \omega^i+\gamma}{c_i+\beta S_{\sigma_c}(\omega^i)+\gamma} = 1$

In [ ]:
omega = Fp.primitive_root_of_unity(n)
roots = Fp([omega**i for i in range(n)])

k1 = 2
k2 = 3

c_roots1 = roots
c_roots2 = roots * k1
c_roots3 = roots * k2

c_roots = np.concatenate((c_roots1, c_roots2, c_roots3))

sigma1 = Fp([c_roots[sigma[i]] for i in range(0, n)])
sigma2 = Fp([c_roots[sigma[i + n]] for i in range(0, n)])
sigma3 = Fp([c_roots[sigma[i + 2 * n]] for i in range(0, n)])

acc = Fp(1)
for i in range(n):
    acc *= (a[i] + beta * c_roots1[i] + gamma) / (a[i] + beta * sigma1[i] + gamma)
    acc *= (b[i] + beta * c_roots2[i] + gamma) / (b[i] + beta * sigma2[i] + gamma)
    acc *= (c[i] + beta * c_roots3[i] + gamma) / (c[i] + beta * sigma3[i] + gamma)

assert acc == 1, "Accumulator is not equal to 1"

### Converting to polynomial

#### Equality of sets

Given:

Random $\gamma$

$A = \{a_0, a_1, ..., a_{k-1}\}$

$B = \{b_0, b_1, ..., b_{k-1}\}$

$A$ is equal to $B$ ($A$ and $B$ have the same elements despite on the order) with high probability if:

$\prod_{i=0}^{k-1} (a_i + \gamma) = \prod_{i=0}^{k-1} (b_i + \gamma)$

Also this equality can be checked by polynomials.

Find roots of unity:

$H: \{1, \omega, ..., \omega^{k-1}\}$

Build polynomials $f(x)$, $g(x)$ that interpolate the following values:

$f(x): \omega^i \to {a_i + \gamma}$

$g(x): \omega^i \to {b_i + \gamma}$

Build polynomial $Z(x)$ that interpolates the following values:

$Z(x): \omega^i \to \left\{ 1, \frac{a_0 + \gamma}{b_0 + \gamma}, \frac{a_0 + \gamma}{b_0 + \gamma} \frac{a_1 + \gamma}{b_1 + \gamma}, ..., \prod_{j=0}^{k-2}\frac{a_j + \gamma}{b_j + \gamma} \right\}$

$Z(x)$ can be represented as a vector of coefficients:

$Z(x) = \sum_{i=0}^{k-1}z_i x^i = \left\langle z_0, z_1, ..., z_{k-1} \right\rangle$

Build polynomial $Z(\omega x)$:

$Z(\omega x) = \sum_{i=0}^{k-1} z_i\omega \cdot x^i = \left\langle z_0, \omega z_1, \omega^2 z_2, ..., \omega^{k-1}z_{k-1} \right\rangle$

---

Sets $A$ and $B$ are equal if and only if there's a polynomial $Z(x)$ such that:

$Z(\omega^0) = 1$

$Z(h)f(h) = g(h)Z(\omega h), h \in H$

---

Check of correctness:

$Z(h)\frac{f(h)}{g(h)} = Z(\omega h)$

$h = \omega^0: Z(\omega^0)\frac{f(\omega^{0})}{g(\omega^{0})} = \color{Cyan} 1\cdot\frac{a_0 + \gamma}{b_0+\gamma} \color{defaultcolor} = Z(\omega \omega^0) = Z(\omega^1) \color{Cyan} = \frac{a_0 + \gamma}{b_0+\gamma}$

$h = \omega^1: Z(\omega^1)\frac{f(\omega^{1})}{g(\omega^{1})} \color{Cyan} = \frac{a_0 + \gamma}{b_0+\gamma} \cdot \frac{a_1 + \gamma}{b_1+\gamma} \color{defaultcolor} = Z(\omega \omega^1) = Z(\omega^2) \color{Cyan} = \frac{a_0 + \gamma}{b_0+\gamma}\frac{a_1 + \gamma}{b_1+\gamma}$

$h = \omega^2: Z(\omega^2)\frac{f(\omega^{2})}{g(\omega^{2})} = Z(\omega \omega^2) = Z(\omega^3)$

...

$h = \omega^{k-1}: Z(\omega^{k-1})\frac{f(\omega^{k-1})}{g(\omega^{k-1})} = Z(\omega \omega^{k-1}) = Z(\omega^k) = Z(\omega^0) = 1$

In [ ]:
k = 8

p = 97
Fp = galois.GF(p)

a = Fp.Random(k)
b = Fp(np.random.permutation(a))

gamma = Fp(42)

a_prod = Fp(1)
b_prod = Fp(1)

for i in range(0, k):
    a_prod *= a[i] + gamma
    b_prod *= b[i] + gamma

assert a_prod == b_prod, "a and b are not equal"

In [ ]:
def shift_poly(poly: galois.Poly, omega: galois.FieldArray):
    coeffs = poly.coefficients(order="asc")
    coeffs = [c * omega**i for i, c in enumerate(coeffs)]
    return galois.Poly(coeffs, order="asc", field=poly.field)

# b[0] = FP.Random()

omega = Fp.primitive_root_of_unity(k)
roots = Fp([omega**i for i in range(k)])

F = galois.lagrange_poly(roots, Fp([_a + gamma for _a in a]))
G = galois.lagrange_poly(roots, Fp([_b + gamma for _b in b]))

for i in range(0, k):
    assert F(roots[i]) == a[i] + gamma, f"F(roots[{i}]) != a[{i}] + gamma"
    assert G(roots[i]) == b[i] + gamma, f"G(roots[{i}]) != b[{i}] + gamma"

acc = [Fp(1)]
for i in range(0, k - 1):
    acc.append(acc[-1] * (a[i] + gamma) / (b[i] + gamma))

Z = galois.lagrange_poly(roots, Fp(acc))

for i in range(0, k):
    assert Z(roots[i]) == acc[i], f"Z(roots[{i}]) != acc[{i}]"

Z_omega = shift_poly(Z, omega)

for i in range(0, k-1):    
    assert Z(roots[i + 1]) == Z_omega(roots[i]), "Z(roots[0]) != Z_omega(roots[1])"

for i in range(0, k):
    assert Z(roots[i]) * F(roots[i]) == G(roots[i]) * Z_omega(roots[i]), "Z * F != G * Z_omega"

#### Applying

In [ ]:
n = 4

p = 97
Fp = galois.GF(p)

a = Fp([4, 16, 4, 68])
b = Fp([4, 4, 64, 68])
c = Fp([16, 64, 68, 73])

ai = range(0, n)
bi = range(n, 2*n)
ci = range(2*n, 3*n)

sigma = {
    ai[0]: ai[2], ai[1]: ci[0], ai[2]: bi[0], ai[3]: ci[2],
    bi[0]: bi[1], bi[1]: ai[0], bi[2]: ci[1], bi[3]: bi[3],
    ci[0]: ai[1], ci[1]: bi[2], ci[2]: ai[3], ci[3]: ci[3],
}

gamma = Fp(10)
beta = Fp(4)
omega = Fp.primitive_root_of_unity(n)
roots = Fp([omega**i for i in range(n)])

k1 = 2
k2 = 3

c1_roots = roots
c2_roots = roots * k1
c3_roots = roots * k2

c_roots = np.concatenate((c1_roots, c2_roots, c3_roots))

sigma1 = Fp([c_roots[sigma[i]] for i in range(0, n)])
sigma2 = Fp([c_roots[sigma[i + n]] for i in range(0, n)])
sigma3 = Fp([c_roots[sigma[i + 2 * n]] for i in range(0, n)])

acc = Fp(1)
for i in range(n):
    acc *= (a[i] + beta * c_roots1[i] + gamma) / (a[i] + beta * sigma1[i] + gamma)
    acc *= (b[i] + beta * c_roots2[i] + gamma) / (b[i] + beta * sigma2[i] + gamma)
    acc *= (c[i] + beta * c_roots3[i] + gamma) / (c[i] + beta * sigma3[i] + gamma)

assert acc == 1, "Accumulator is not equal to 1"

In [ ]:
def to_galois_array(vector, field):
    return field([x % field.order for x in vector])

def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field) if isinstance(v, list) else v
    return galois.lagrange_poly(x, y)

S1 = to_poly(roots, sigma1, Fp)
S2 = to_poly(roots, sigma2, Fp)
S3 = to_poly(roots, sigma3, Fp)

I1 = to_poly(roots, c1_roots, Fp)
I2 = to_poly(roots, c2_roots, Fp)
I3 = to_poly(roots, c3_roots, Fp)

A = to_poly(roots, a, Fp)
B = to_poly(roots, b, Fp)
C = to_poly(roots, c, Fp)

_F = (A + I1 * beta + gamma) * (B + I2 * beta + gamma) * (C + I3 * beta + gamma)
_G = (A + S1 * beta + gamma) * (B + S2 * beta + gamma) * (C + S3 * beta + gamma)

acc = [Fp(1)]
for i in range(0, n - 1):
    acc.append(acc[-1] * (_F(roots[i]) / _G(roots[i])))

Z = galois.lagrange_poly(roots, Fp(acc))

assert Z(roots[0]) == 1, "Z(roots[0]) != 1"

Z_omega = shift_poly(Z, omega)

for i in range(0, n-1):
    assert Z(roots[i + 1]) == Z_omega(roots[i]), "Z(roots[0]) != Z_omega(roots[1])"

for i in range(0, n):
    assert Z(roots[i]) * _F(roots[i]) == _G(roots[i]) * Z_omega(roots[i]), "Z * F != G * Z_omega"